<a href="https://colab.research.google.com/github/nikhil-iitb/DerivativePricing_MarketMaking/blob/main/Copy_of_GNN_OptionPricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import keras
from keras.layers import Input, Dense, Activation, concatenate, Multiply, multiply
from keras.models import Sequential, Model
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
file_path = '/content/drive/MyDrive/BTechProject/Data/data.csv'
df = pd.read_csv(file_path)
df.head()

Mounted at /content/drive


,Unnamed: 0,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility
0,0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,2.0,1.163095,0.053646,0.022956
1,1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,2.0,0.997845,0.053646,0.114784
2,2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,2.0,0.997223,0.053646,0.106823
3,3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,2.0,1.002110,0.053646,0.110336
4,4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,2.0,0.992831,0.053646,0.110336


In [ ]:
option = df

In [ ]:
option.dropna()
option['days_to_maturity'] = option['days_to_maturity']/365

In [ ]:
# Transformed Market Price  y=c*exp(rt)/St
option['transform']=option['contract_price']*np.exp(option['rate']*option['days_to_maturity'])/option['underlyings_price']
option.head()

,Unnamed: 0,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,transform,inverse_moneyness,moneyness_class
0,0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,0.005479,1.163095,0.053646,0.022956,0.146409,0.859775,itm
1,1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,0.005479,0.997845,0.053646,0.114784,0.003108,1.002160,atm
2,2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,0.005479,0.997223,0.053646,0.106823,0.002783,1.002785,atm
3,3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,0.005479,1.002110,0.053646,0.110336,0.005523,0.997894,atm
4,4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,0.005479,0.992831,0.053646,0.110336,0.001399,1.007220,atm


In [ ]:
option['inverse_moneyness']=1/option['moneyness']

In [ ]:
option[option['callput']=='C'].count()

Unnamed: 0           107499
optionid             107499
securityid           107499
strike               107499
callput              107499
date_traded          107499
contract_price       107499
market_price         107499
underlyings_price    107499
contract_volume      107499
days_to_maturity     107499
moneyness            107499
rate                 107499
volatility           107499
transform            107499
inverse_moneyness    107499
moneyness_class      107499
dtype: int64

In [ ]:
option['moneyness_class'] = np.where(option['moneyness']>1.05,'itm','atm')
option['moneyness_class'] = np.where(option['moneyness']<0.97,'otm',option['moneyness_class'])

In [ ]:
option.head()

,Unnamed: 0,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,transform,inverse_moneyness,moneyness_class
0,0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,0.005479,1.163095,0.053646,0.022956,0.146409,0.859775,itm
1,1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,0.005479,0.997845,0.053646,0.114784,0.003108,1.002160,atm
2,2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,0.005479,0.997223,0.053646,0.106823,0.002783,1.002785,atm
3,3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,0.005479,1.002110,0.053646,0.110336,0.005523,0.997894,atm
4,4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,0.005479,0.992831,0.053646,0.110336,0.001399,1.007220,atm


In [ ]:
call_option = option
call_option['tau_class'] = np.where(call_option['days_to_maturity']>0.2,'longterm','midterm')
call_option['tau_class'] = np.where(call_option['days_to_maturity']<0.1,'shortterm',call_option['tau_class'])

In [ ]:
call_option[call_option['tau_class']=='midterm'].count()

Unnamed: 0           28259
optionid             28259
securityid           28259
strike               28259
callput              28259
date_traded          28259
contract_price       28259
market_price         28259
underlyings_price    28259
contract_volume      28259
days_to_maturity     28259
moneyness            28259
rate                 28259
volatility           28259
transform            28259
inverse_moneyness    28259
moneyness_class      28259
tau_class            28259
dtype: int64

In [ ]:
call_option.head()

,Unnamed: 0,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,transform,inverse_moneyness,moneyness_class,tau_class
0,0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,0.005479,1.163095,0.053646,0.022956,0.146409,0.859775,itm,shortterm
1,1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,0.005479,0.997845,0.053646,0.114784,0.003108,1.002160,atm,shortterm
2,2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,0.005479,0.997223,0.053646,0.106823,0.002783,1.002785,atm,shortterm
3,3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,0.005479,1.002110,0.053646,0.110336,0.005523,0.997894,atm,shortterm
4,4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,0.005479,0.992831,0.053646,0.110336,0.001399,1.007220,atm,shortterm


In [ ]:
call_option['year'] = call_option['date_traded'].str.split('-').str[0]
call_option.head()
print(call_option['year'].value_counts())

2006    20160
2007    12763
2015     8814
2020     8619
2016     8286
2019     8211
2008     7928
2017     6985
2018     6456
2014     5805
2021     3500
2012     3440
2013     3391
2011     2933
2010      144
2009       64
Name: year, dtype: int64


In [ ]:
def Multi_pricing(option_data,output_df):
    years_performance = []
    years_data = pd.DataFrame()
    years = list(option_data['year'].unique())
    print(years)
    years.remove('2009') # too few samples
    years.remove('2010')
    yearly_data = [option_data[option_data['year'] == i] for i in years]
    for yearwise_data in yearly_data: #different years
        yearwise_data_train = yearwise_data.iloc[:int(len(yearwise_data)/2),:]
        yearwise_data_validation = yearwise_data.iloc[int(len(yearwise_data)/2):int(len(yearwise_data)*3/4),:]
        yearwise_data_test= yearwise_data.iloc[int(len(yearwise_data)*3/4):,:]

        # build multi model

        # create 9 single models
        #1
        # model_1_left=Sequential()
        # model_1_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_1_left.add(Activation('softplus'))   #softplus

        # model_1_right = Sequential()
        # model_1_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_1_right.add(Activation('sigmoid')) #sigmoid

        # model_1 = Sequential()
        # model_1.add(multiply([model_1_left.output,model_1_right.output]))  # multiplication gate

        # model_1.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_1.add(Activation('linear'))  #linear

        # Define the left submodel
        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_1 = Model(inputs=[input_left, input_right], outputs=output)

        #2
        # model_2_left=Sequential()
        # model_2_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_2_left.add(Activation('softplus'))   #softplus

        # model_2_right = Sequential()
        # model_2_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_2_right.add(Activation('sigmoid')) #sigmoid

        # model_2 = Sequential()
        # model_2.add(concatenate([model_2_left,model_2_right], axis=-1))  # multiplication gate

        # model_2.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_2.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_2 = Model(inputs=[input_left, input_right], outputs=output)

        #3
        # model_3_left=Sequential()
        # model_3_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_3_left.add(Activation('softplus'))   #softplus

        # model_3_right = Sequential()
        # model_3_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_3_right.add(Activation('sigmoid')) #sigmoid

        # model_3 = Sequential()
        # model_3.add(concatenate([model_3_left,model_3_right], axis=-1))  # multiplication gate

        # model_3.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_3.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_3 = Model(inputs=[input_left, input_right], outputs=output)

        #4
        # model_4_left=Sequential()
        # model_4_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_4_left.add(Activation('softplus'))   #softplus

        # model_4_right = Sequential()
        # model_4_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_4_right.add(Activation('sigmoid')) #sigmoid

        # model_4 = Sequential()
        # model_4.add(concatenate([model_4_left,model_4_right], axis=-1))  # multiplication gate

        # model_4.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_4.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_4 = Model(inputs=[input_left, input_right], outputs=output)

        #5
        # model_5_left=Sequential()
        # model_5_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_5_left.add(Activation('softplus'))   #softplus

        # model_5_right = Sequential()
        # model_5_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_5_right.add(Activation('sigmoid')) #sigmoid

        # model_5 = Sequential()
        # model_5.add(concatenate([model_5_left,model_5_right], axis=-1))  # multiplication gate

        # model_5.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_5.add(Activation('linear'))  #linear
        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_5 = Model(inputs=[input_left, input_right], outputs=output)

        #6
        # model_6_left=Sequential()
        # model_6_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_6_left.add(Activation('softplus'))   #softplus

        # model_6_right = Sequential()
        # model_6_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_6_right.add(Activation('sigmoid')) #sigmoid

        # model_6 = Sequential()
        # model_6.add(concatenate([model_6_left,model_6_right], axis=-1))  # multiplication gate

        # model_6.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_6.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_6 = Model(inputs=[input_left, input_right], outputs=output)

        #7
        # model_7_left=Sequential()
        # model_7_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_7_left.add(Activation('softplus'))   #softplus

        # model_7_right = Sequential()
        # model_7_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_7_right.add(Activation('sigmoid')) #sigmoid

        # model_7 = Sequential()
        # model_7.add(concatenate([model_7_left,model_7_right], axis=-1))  # multiplication gate

        # model_7.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_7.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_7 = Model(inputs=[input_left, input_right], outputs=output)


        #8
        # model_8_left=Sequential()
        # model_8_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_8_left.add(Activation('softplus'))   #softplus

        # model_8_right = Sequential()
        # model_8_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_8_right.add(Activation('sigmoid')) #sigmoid

        # model_8 = Sequential()
        # model_8.add(concatenate([model_8_left,model_8_right], axis=-1))  # multiplication gate

        # model_8.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_8.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_8 = Model(inputs=[input_left, input_right], outputs=output)

        #9
        # model_9_left=Sequential()
        # model_9_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_9_left.add(Activation('softplus'))   #softplus

        # model_9_right = Sequential()
        # model_9_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_9_right.add(Activation('sigmoid')) #sigmoid

        # model_9 = Sequential()
        # model_9.add(concatenate([model_9_left,model_9_right], axis=-1))  # multiplication gate

        # model_9.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_9.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_9 = Model(inputs=[input_left, input_right], outputs=output)



        # left branch
        inputs = [model.input for model in [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9]]

        # Concatenate the outputs of the models
        concatenated_output = Concatenate(axis=-1)([model.output for model in [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9]])

        # Create a new model with the concatenated output
        model_left = Model(inputs=inputs, outputs=concatenated_output)
        # right branch
        model_right = Sequential()
        model_right.add(Dense(5, activation='sigmoid', input_dim=2))
        model_right.add(Dense(9, activation='softmax'))

        # whole model
        # model = Sequential()
        # model.add(concatenate([model_left,model_right], axis=-1))
        model_left_output = model_left.output
        model_right_output = model_right.output

        # Define a Lambda layer to compute the dot product
        dot_product = Dot(axes=-1)([model_left_output, model_right_output])

        # Create a new model with the Lambda layer
        model = Model(inputs=[model_left.input, model_right.input], outputs=dot_product)
        model.compile(loss='mean_squared_error',
                              optimizer=Adam(),
                              metrics=['mae'])


        # normalize
        sc1 = StandardScaler() # fit training feature1
        sc2 = StandardScaler() # fit training feature2
        sc3 = StandardScaler() # fit validation feature1 and/or target (shared without reuse conflict)
        sc4 = StandardScaler() # fit validation feature2,can reuse
        sc5 = StandardScaler() # fit training target, inverse transform for test prediction
        sc7 = StandardScaler()
        sc8 = StandardScaler()   # can reuse

        print('----------year = ',yearwise_data['year'],'-----------')
        # early stopping
        es = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5, verbose=2, mode='auto')

        # fit
        X1_train=sc1.fit_transform(-yearwise_data_train[['inverse_moneyness']])
        X2_train=sc2.fit_transform(yearwise_data_train[['days_to_maturity']])
        X1_validation=sc3.fit_transform(-yearwise_data_validation[['inverse_moneyness']])
        X2_validation=sc4.fit_transform(yearwise_data_validation[['days_to_maturity']])
        right_train=sc7.fit_transform(yearwise_data_train[['inverse_moneyness','days_to_maturity']])
        right_validation=sc8.fit_transform(yearwise_data_validation[['inverse_moneyness','days_to_maturity']])

        history = model.fit([X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,right_train],sc5.fit_transform(yearwise_data_train[['transform']]),batch_size=50,epochs=100,verbose=2,validation_data=([X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,right_validation],sc3.fit_transform(yearwise_data_validation[['transform']])),callbacks=[es])

        # predict
        X1_test=sc1.transform(-yearwise_data_test[['inverse_moneyness']])
        X2_test=sc2.transform(yearwise_data_test[['days_to_maturity']])
        right_test=sc7.transform(yearwise_data_test[['inverse_moneyness','days_to_maturity']])

        prediction = model.predict([X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,right_test])

        # percentage mean-squared-error
        yearwise_data_test_copy = yearwise_data_test.copy()
        yearwise_data_test_copy['temp'] = sc5.inverse_transform(prediction)  # inverse transform
        yearwise_data_test_copy['Multi_price']=yearwise_data_test_copy['temp']*yearwise_data_test['contract_price']*np.exp(-yearwise_data_test['rate']*yearwise_data_test['days_to_maturity'])
        yearwise_data_test_copy['Multi_square_error'] = ((yearwise_data_test['contract_price']-yearwise_data_test_copy['Multi_price'])/yearwise_data_test['contract_price'])**2
        yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)

        years_performance.append(yearwise_data_test_copy['Multi_square_error'].mean())
        years_data = years_data.append(yearwise_data_test_copy)

    # aggregete
    performance.append(years_performance)
    output_df = output_df.append(years_data)
    print('percentage mean square error is',np.mean(years_performance))
    return output_df

In [ ]:
performance = []
Multi_output = pd.DataFrame()

# test
# call
Multi_output = Multi_pricing(call_option,Multi_output)

['2006', '2007', '2020', '2013', '2017', '2012', '2008', '2010', '2014', '2009', '2011', '2021', '2016', '2015', '2018', '2019']
----------year =  0        2006
1        2006
2        2006
3        2006
4        2006
         ... 
26958    2006
26959    2006
26960    2006
26961    2006
26962    2006
Name: year, Length: 20160, dtype: object -----------
Epoch 1/100
202/202 - 12s - loss: 0.9101 - mae: 0.8116 - val_loss: 0.8107 - val_mae: 0.7829 - 12s/epoch - 60ms/step
Epoch 2/100
202/202 - 2s - loss: 0.7089 - mae: 0.7386 - val_loss: 0.5275 - val_mae: 0.6625 - 2s/epoch - 11ms/step
Epoch 3/100
202/202 - 3s - loss: 0.5263 - mae: 0.6476 - val_loss: 0.3773 - val_mae: 0.5602 - 3s/epoch - 13ms/step
Epoch 4/100
202/202 - 3s - loss: 0.4646 - mae: 0.5994 - val_loss: 0.3352 - val_mae: 0.5242 - 3s/epoch - 15ms/step
Epoch 5/100
202/202 - 2s - loss: 0.4409 - mae: 0.5770 - val_loss: 0.3138 - val_mae: 0.5046 - 2s/epoch - 11ms/step
Epoch 6/100
202/202 - 2s - loss: 0.4254 - mae: 0.5640 - val_loss: 0.3056 -

<ipython-input-119-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-119-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  80       2007
81       2007
82       2007
83       2007
84       2007
         ... 
65166    2007
65167    2007
65223    2007
65224    2007
65225    2007
Name: year, Length: 12763, dtype: object -----------
Epoch 1/100
128/128 - 11s - loss: 1.0699 - mae: 0.8983 - val_loss: 1.0297 - val_mae: 0.8871 - 11s/epoch - 88ms/step
Epoch 2/100
128/128 - 1s - loss: 0.9554 - mae: 0.8459 - val_loss: 0.9606 - val_mae: 0.8559 - 1s/epoch - 11ms/step
Epoch 3/100
128/128 - 2s - loss: 0.8483 - mae: 0.7966 - val_loss: 0.8824 - val_mae: 0.8198 - 2s/epoch - 13ms/step
Epoch 4/100
128/128 - 1s - loss: 0.7072 - mae: 0.7163 - val_loss: 0.7727 - val_mae: 0.7612 - 1s/epoch - 11ms/step
Epoch 5/100
128/128 - 2s - loss: 0.5904 - mae: 0.6357 - val_loss: 0.6977 - val_mae: 0.7162 - 2s/epoch - 12ms/step
Epoch 6/100
128/128 - 1s - loss: 0.5315 - mae: 0.5916 - val_loss: 0.6639 - val_mae: 0.6956 - 1s/epoch - 11ms/step
Epoch 7/100
128/128 - 2s - loss: 0.5055 - mae: 0.5703 - val_loss: 0.6433 - val_mae: 0.679

<ipython-input-119-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-119-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  1069      2020
1070      2020
1071      2020
1072      2020
1073      2020
          ... 
107363    2020
107370    2020
107371    2020
107497    2020
107498    2020
Name: year, Length: 8619, dtype: object -----------
Epoch 1/100
87/87 - 11s - loss: 1.0425 - mae: 0.7865 - val_loss: 1.0062 - val_mae: 0.8193 - 11s/epoch - 123ms/step
Epoch 2/100
87/87 - 1s - loss: 1.0099 - mae: 0.7626 - val_loss: 0.9879 - val_mae: 0.8057 - 1s/epoch - 13ms/step
Epoch 3/100
87/87 - 1s - loss: 0.9966 - mae: 0.7517 - val_loss: 0.9786 - val_mae: 0.7988 - 1s/epoch - 12ms/step
Epoch 4/100
87/87 - 1s - loss: 0.9897 - mae: 0.7460 - val_loss: 0.9719 - val_mae: 0.7945 - 1s/epoch - 12ms/step
Epoch 5/100
87/87 - 1s - loss: 0.9847 - mae: 0.7417 - val_loss: 0.9662 - val_mae: 0.7914 - 978ms/epoch - 11ms/step
Epoch 6/100
87/87 - 1s - loss: 0.9803 - mae: 0.7402 - val_loss: 0.9592 - val_mae: 0.7890 - 987ms/epoch - 11ms/step
Epoch 7/100
87/87 - 1s - loss: 0.9759 - mae: 0.7386 - val_loss: 0.9522 - val_mae: 0.

...

<ipython-input-119-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-119-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  38726     2018
38727     2018
38728     2018
38729     2018
38730     2018
          ... 
92748     2018
92753     2018
92754     2018
92979     2018
107132    2018
Name: year, Length: 6456, dtype: object -----------
Epoch 1/100
65/65 - 9s - loss: 1.0252 - mae: 0.8881 - val_loss: 1.0301 - val_mae: 0.8942 - 9s/epoch - 134ms/step
Epoch 2/100
65/65 - 1s - loss: 0.9770 - mae: 0.8640 - val_loss: 0.9959 - val_mae: 0.8759 - 1s/epoch - 16ms/step
Epoch 3/100
65/65 - 1s - loss: 0.9483 - mae: 0.8489 - val_loss: 0.9753 - val_mae: 0.8649 - 1s/epoch - 18ms/step
Epoch 4/100
65/65 - 1s - loss: 0.9267 - mae: 0.8376 - val_loss: 0.9591 - val_mae: 0.8569 - 1s/epoch - 16ms/step
Epoch 5/100
65/65 - 1s - loss: 0.9058 - mae: 0.8280 - val_loss: 0.9434 - val_mae: 0.8502 - 943ms/epoch - 15ms/step
Epoch 6/100
65/65 - 1s - loss: 0.8817 - mae: 0.8173 - val_loss: 0.9249 - val_mae: 0.8428 - 803ms/epoch - 12ms/step
Epoch 7/100
65/65 - 1s - loss: 0.8515 - mae: 0.8042 - val_loss: 0.8992 - val_mae: 0.83

<ipython-input-119-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-119-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  38766     2019
38767     2019
38768     2019
38769     2019
38770     2019
          ... 
107282    2019
107283    2019
107284    2019
107495    2019
107496    2019
Name: year, Length: 8211, dtype: object -----------
Epoch 1/100
83/83 - 10s - loss: 1.0543 - mae: 0.8796 - val_loss: 1.0223 - val_mae: 0.8715 - 10s/epoch - 126ms/step
Epoch 2/100
83/83 - 1s - loss: 0.9977 - mae: 0.8509 - val_loss: 0.9850 - val_mae: 0.8490 - 1s/epoch - 12ms/step
Epoch 3/100
83/83 - 1s - loss: 0.9690 - mae: 0.8355 - val_loss: 0.9624 - val_mae: 0.8349 - 1s/epoch - 12ms/step
Epoch 4/100
83/83 - 1s - loss: 0.9497 - mae: 0.8255 - val_loss: 0.9449 - val_mae: 0.8263 - 988ms/epoch - 12ms/step
Epoch 5/100
83/83 - 1s - loss: 0.9323 - mae: 0.8175 - val_loss: 0.9266 - val_mae: 0.8189 - 984ms/epoch - 12ms/step
Epoch 6/100
83/83 - 1s - loss: 0.9129 - mae: 0.8092 - val_loss: 0.9046 - val_mae: 0.8114 - 982ms/epoch - 12ms/step
Epoch 7/100
83/83 - 1s - loss: 0.8900 - mae: 0.7998 - val_loss: 0.8796 - val_mae:

<ipython-input-119-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-119-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)
<ipython-input-119-81b9608060d2>:397: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(years_data)


In [ ]:
call_option.head()

,Unnamed: 0,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,transform,inverse_moneyness,moneyness_class,tau_class,year
0,0,150034236.0,504569.0,0.42,C,2006-10-18,0.0715,0.07025,0.488500,5.0,0.005479,1.163095,0.053646,0.022956,0.146409,0.859775,itm,shortterm,2006
1,1,150247468.0,504880.0,40.00,C,2006-10-18,0.1240,0.12250,39.913799,56137.0,0.005479,0.997845,0.053646,0.114784,0.003108,1.002160,atm,shortterm,2006
2,2,150255000.0,506496.0,62.00,C,2006-10-18,0.1720,0.17400,61.827798,27369.0,0.005479,0.997223,0.053646,0.106823,0.002783,1.002785,atm,shortterm,2006
3,3,150255496.0,506497.0,53.50,C,2006-10-18,0.2960,0.26550,53.612900,1224.0,0.005479,1.002110,0.053646,0.110336,0.005523,0.997894,atm,shortterm,2006
4,4,150255498.0,506497.0,54.00,C,2006-10-18,0.0750,0.06450,53.612900,963.0,0.005479,0.992831,0.053646,0.110336,0.001399,1.007220,atm,shortterm,2006


In [ ]:
mask = ~((df['moneyness_class'] == 'itm') | (df['days_to_maturity'] <= 2/365))
df_filtered = df[mask]

In [ ]:
df_filtered

,Unnamed: 0,optionid,securityid,strike,callput,date_traded,contract_price,market_price,underlyings_price,contract_volume,days_to_maturity,moneyness,rate,volatility,transform,inverse_moneyness,moneyness_class,tau_class,year
27,27,150247466.0,504880.0,39.5,C,2006-10-17,0.222,0.21800,39.495701,8570.0,0.008219,0.999891,0.053746,0.111227,0.005623,1.000109,atm,shortterm,2006
28,28,150254998.0,506496.0,61.5,C,2006-10-17,0.166,0.17100,61.151001,12174.0,0.008219,0.994325,0.053746,0.102518,0.002716,1.005707,atm,shortterm,2006
29,29,150255000.0,506496.0,62.0,C,2006-10-17,0.053,0.05350,61.151001,27460.0,0.008219,0.986306,0.053746,0.102518,0.000867,1.013884,atm,shortterm,2006
30,30,150255494.0,506497.0,53.0,C,2006-10-17,0.310,0.30050,53.029902,762.0,0.008219,1.000564,0.053746,0.105705,0.005848,0.999436,atm,shortterm,2006
31,31,150255496.0,506497.0,53.5,C,2006-10-17,0.110,0.10150,53.029902,1934.0,0.008219,0.991213,0.053746,0.105705,0.002075,1.008865,atm,shortterm,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107494,108020,175215018.0,782358.0,10.6,C,2021-09-22,0.075,0.08400,10.130000,20.0,0.158904,0.955660,0.001108,0.184646,0.007405,1.046397,otm,midterm,2021
107495,108021,163936749.0,782360.0,11.2,C,2019-03-12,0.092,0.08750,11.097500,50.0,0.104110,0.990848,0.026018,0.093048,0.008313,1.009236,atm,midterm,2019
107496,108022,163936801.0,782360.0,12.0,C,2019-06-21,0.097,0.07850,11.400000,50.0,0.249315,0.950000,0.022493,0.161584,0.008557,1.052632,otm,longterm,2019
107497,108023,165155300.0,782360.0,13.2,C,2020-01-28,0.094,0.08550,12.767500,50.0,0.219178,0.967235,0.017113,0.153323,0.007390,1.033875,otm,longterm,2020


In [ ]:
def Multi_pricing(option_data,output_df):
    years_performance = []
    years_data = pd.DataFrame()
    years = list(option_data['year'].unique())
    print(years)
    years.remove('2009') # too few samples
    years.remove('2010')
    yearly_data = [option_data[option_data['year'] == i] for i in years]
    for yearwise_data in yearly_data: #different years
        yearwise_data_train = yearwise_data.iloc[:int(len(yearwise_data)/2),:]
        yearwise_data_validation = yearwise_data.iloc[int(len(yearwise_data)/2):int(len(yearwise_data)*3/4),:]
        yearwise_data_test= yearwise_data.iloc[int(len(yearwise_data)*3/4):,:]

        # build multi model

        # create 9 single models
        #1
        # model_1_left=Sequential()
        # model_1_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_1_left.add(Activation('softplus'))   #softplus

        # model_1_right = Sequential()
        # model_1_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_1_right.add(Activation('sigmoid')) #sigmoid

        # model_1 = Sequential()
        # model_1.add(multiply([model_1_left.output,model_1_right.output]))  # multiplication gate

        # model_1.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_1.add(Activation('linear'))  #linear

        # Define the left submodel
        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_1 = Model(inputs=[input_left, input_right], outputs=output)

        #2
        # model_2_left=Sequential()
        # model_2_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_2_left.add(Activation('softplus'))   #softplus

        # model_2_right = Sequential()
        # model_2_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_2_right.add(Activation('sigmoid')) #sigmoid

        # model_2 = Sequential()
        # model_2.add(concatenate([model_2_left,model_2_right], axis=-1))  # multiplication gate

        # model_2.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_2.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_2 = Model(inputs=[input_left, input_right], outputs=output)

        #3
        # model_3_left=Sequential()
        # model_3_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_3_left.add(Activation('softplus'))   #softplus

        # model_3_right = Sequential()
        # model_3_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_3_right.add(Activation('sigmoid')) #sigmoid

        # model_3 = Sequential()
        # model_3.add(concatenate([model_3_left,model_3_right], axis=-1))  # multiplication gate

        # model_3.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_3.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_3 = Model(inputs=[input_left, input_right], outputs=output)

        #4
        # model_4_left=Sequential()
        # model_4_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_4_left.add(Activation('softplus'))   #softplus

        # model_4_right = Sequential()
        # model_4_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_4_right.add(Activation('sigmoid')) #sigmoid

        # model_4 = Sequential()
        # model_4.add(concatenate([model_4_left,model_4_right], axis=-1))  # multiplication gate

        # model_4.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_4.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_4 = Model(inputs=[input_left, input_right], outputs=output)

        #5
        # model_5_left=Sequential()
        # model_5_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_5_left.add(Activation('softplus'))   #softplus

        # model_5_right = Sequential()
        # model_5_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_5_right.add(Activation('sigmoid')) #sigmoid

        # model_5 = Sequential()
        # model_5.add(concatenate([model_5_left,model_5_right], axis=-1))  # multiplication gate

        # model_5.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_5.add(Activation('linear'))  #linear
        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_5 = Model(inputs=[input_left, input_right], outputs=output)

        #6
        # model_6_left=Sequential()
        # model_6_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_6_left.add(Activation('softplus'))   #softplus

        # model_6_right = Sequential()
        # model_6_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_6_right.add(Activation('sigmoid')) #sigmoid

        # model_6 = Sequential()
        # model_6.add(concatenate([model_6_left,model_6_right], axis=-1))  # multiplication gate

        # model_6.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_6.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_6 = Model(inputs=[input_left, input_right], outputs=output)

        #7
        # model_7_left=Sequential()
        # model_7_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_7_left.add(Activation('softplus'))   #softplus

        # model_7_right = Sequential()
        # model_7_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_7_right.add(Activation('sigmoid')) #sigmoid

        # model_7 = Sequential()
        # model_7.add(concatenate([model_7_left,model_7_right], axis=-1))  # multiplication gate

        # model_7.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_7.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_7 = Model(inputs=[input_left, input_right], outputs=output)


        #8
        # model_8_left=Sequential()
        # model_8_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_8_left.add(Activation('softplus'))   #softplus

        # model_8_right = Sequential()
        # model_8_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_8_right.add(Activation('sigmoid')) #sigmoid

        # model_8 = Sequential()
        # model_8.add(concatenate([model_8_left,model_8_right], axis=-1))  # multiplication gate

        # model_8.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_8.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_8 = Model(inputs=[input_left, input_right], outputs=output)

        #9
        # model_9_left=Sequential()
        # model_9_left.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_9_left.add(Activation('softplus'))   #softplus

        # model_9_right = Sequential()
        # model_9_right.add(Dense(5, input_shape=(1,),kernel_constraint=non_neg()))
        # model_9_right.add(Activation('sigmoid')) #sigmoid

        # model_9 = Sequential()
        # model_9.add(concatenate([model_9_left,model_9_right], axis=-1))  # multiplication gate

        # model_9.add(Dense(1,use_bias=False,kernel_constraint=non_neg()))
        # model_9.add(Activation('linear'))  #linear

        input_left = Input(shape=(1,))
        left = Dense(5, kernel_constraint=non_neg())(input_left)
        left = Activation('softplus')(left)

        # Define the right submodel
        input_right = Input(shape=(1,))
        right = Dense(5, kernel_constraint=non_neg())(input_right)
        right = Activation('sigmoid')(right)

        # Multiply the left and right submodels
        merged = Multiply()([left, right])

        # Define the final model
        output = Dense(1, use_bias=False, kernel_constraint=non_neg())(merged)
        output = Activation('linear')(output)

        # Create the model
        model_9 = Model(inputs=[input_left, input_right], outputs=output)



        # left branch
        inputs = [model.input for model in [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9]]

        # Concatenate the outputs of the models
        concatenated_output = Concatenate(axis=-1)([model.output for model in [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9]])

        # Create a new model with the concatenated output
        model_left = Model(inputs=inputs, outputs=concatenated_output)
        # right branch
        model_right = Sequential()
        model_right.add(Dense(5, activation='sigmoid', input_dim=2))
        model_right.add(Dense(9, activation='softmax'))

        # whole model
        # model = Sequential()
        # model.add(concatenate([model_left,model_right], axis=-1))
        model_left_output = model_left.output
        model_right_output = model_right.output

        # Define a Lambda layer to compute the dot product
        dot_product = Dot(axes=-1)([model_left_output, model_right_output])

        # Create a new model with the Lambda layer
        model = Model(inputs=[model_left.input, model_right.input], outputs=dot_product)
        model.compile(loss='mean_squared_error',
                              optimizer=Adam(),
                              metrics=['mae'])


        # normalize
        sc1 = StandardScaler() # fit training feature1
        sc2 = StandardScaler() # fit training feature2
        sc3 = StandardScaler() # fit validation feature1 and/or target (shared without reuse conflict)
        sc4 = StandardScaler() # fit validation feature2,can reuse
        sc5 = StandardScaler() # fit training target, inverse transform for test prediction
        sc7 = StandardScaler()
        sc8 = StandardScaler()   # can reuse

        print('----------year = ',yearwise_data['year'],'-----------')
        # early stopping
        es = keras.callbacks.EarlyStopping(monitor='val_loss',min_delta=0,patience=5, verbose=2, mode='auto')

        # fit
        X1_train=sc1.fit_transform(-yearwise_data_train[['inverse_moneyness']])
        X2_train=sc2.fit_transform(yearwise_data_train[['days_to_maturity']])
        X1_validation=sc3.fit_transform(-yearwise_data_validation[['inverse_moneyness']])
        X2_validation=sc4.fit_transform(yearwise_data_validation[['days_to_maturity']])
        right_train=sc7.fit_transform(yearwise_data_train[['inverse_moneyness','days_to_maturity']])
        right_validation=sc8.fit_transform(yearwise_data_validation[['inverse_moneyness','days_to_maturity']])

        history = model.fit([X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,X1_train,X2_train,right_train],sc5.fit_transform(yearwise_data_train[['transform']]),batch_size=50,epochs=100,verbose=2,validation_data=([X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,X1_validation,X2_validation,right_validation],sc3.fit_transform(yearwise_data_validation[['transform']])),callbacks=[es])

        # predict
        X1_test=sc1.transform(-yearwise_data_test[['inverse_moneyness']])
        X2_test=sc2.transform(yearwise_data_test[['days_to_maturity']])
        right_test=sc7.transform(yearwise_data_test[['inverse_moneyness','days_to_maturity']])

        prediction = model.predict([X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,X1_test,X2_test,right_test])

        # percentage mean-squared-error
        yearwise_data_test_copy = yearwise_data_test.copy()
        yearwise_data_test_copy['temp'] = sc5.inverse_transform(prediction)  # inverse transform
        yearwise_data_test_copy['Multi_price']=yearwise_data_test_copy['temp']*yearwise_data_test['contract_price']*np.exp(-yearwise_data_test['rate']*yearwise_data_test['days_to_maturity'])
        yearwise_data_test_copy['Multi_square_error'] = ((yearwise_data_test['contract_price']-yearwise_data_test_copy['Multi_price'])/yearwise_data_test['contract_price'])**2
        yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)

        years_performance.append(yearwise_data_test_copy['Multi_square_error'].mean())
        years_data = years_data.append(yearwise_data_test_copy)

    # aggregete
    performance.append(years_performance)
    output_df = output_df.append(years_data)
    print('percentage mean square error is',np.mean(years_performance))
    return output_df

In [ ]:
performance = []
Multi_output2 = pd.DataFrame()

# test
# call
Multi_output = Multi_pricing(df_filtered,Multi_output2)

['2006', '2007', '2020', '2013', '2017', '2008', '2010', '2014', '2009', '2011', '2012', '2021', '2016', '2015', '2018', '2019']
----------year =  27       2006
28       2006
29       2006
30       2006
31       2006
         ... 
26958    2006
26959    2006
26960    2006
26961    2006
26962    2006
Name: year, Length: 18113, dtype: object -----------
Epoch 1/100
182/182 - 11s - loss: 0.9876 - mae: 0.8538 - val_loss: 0.9113 - val_mae: 0.8297 - 11s/epoch - 61ms/step
Epoch 2/100
182/182 - 3s - loss: 0.8429 - mae: 0.8015 - val_loss: 0.6968 - val_mae: 0.7535 - 3s/epoch - 16ms/step
Epoch 3/100
182/182 - 3s - loss: 0.6144 - mae: 0.7112 - val_loss: 0.4554 - val_mae: 0.6206 - 3s/epoch - 14ms/step
Epoch 4/100
182/182 - 2s - loss: 0.4491 - mae: 0.6067 - val_loss: 0.3557 - val_mae: 0.5407 - 2s/epoch - 12ms/step
Epoch 5/100
182/182 - 2s - loss: 0.3860 - mae: 0.5549 - val_loss: 0.3228 - val_mae: 0.5123 - 2s/epoch - 12ms/step
Epoch 6/100
182/182 - 2s - loss: 0.3577 - mae: 0.5288 - val_loss: 0.3069 -

<ipython-input-125-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-125-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  81       2007
82       2007
83       2007
84       2007
87       2007
         ... 
65166    2007
65167    2007
65223    2007
65224    2007
65225    2007
Name: year, Length: 11487, dtype: object -----------
Epoch 1/100
115/115 - 9s - loss: 1.0152 - mae: 0.8644 - val_loss: 1.0041 - val_mae: 0.8599 - 9s/epoch - 82ms/step
Epoch 2/100
115/115 - 2s - loss: 0.9285 - mae: 0.8248 - val_loss: 0.9519 - val_mae: 0.8369 - 2s/epoch - 13ms/step
Epoch 3/100
115/115 - 2s - loss: 0.8526 - mae: 0.7916 - val_loss: 0.8919 - val_mae: 0.8138 - 2s/epoch - 19ms/step
Epoch 4/100
115/115 - 1s - loss: 0.7624 - mae: 0.7472 - val_loss: 0.8250 - val_mae: 0.7834 - 1s/epoch - 13ms/step
Epoch 5/100
115/115 - 1s - loss: 0.6850 - mae: 0.7003 - val_loss: 0.7712 - val_mae: 0.7562 - 1s/epoch - 12ms/step
Epoch 6/100
115/115 - 1s - loss: 0.6277 - mae: 0.6616 - val_loss: 0.7311 - val_mae: 0.7351 - 1s/epoch - 12ms/step
Epoch 7/100
115/115 - 1s - loss: 0.5875 - mae: 0.6336 - val_loss: 0.7085 - val_mae: 0.7234 

<ipython-input-125-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-125-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  1069      2020
1070      2020
1071      2020
1072      2020
1073      2020
          ... 
107363    2020
107370    2020
107371    2020
107497    2020
107498    2020
Name: year, Length: 8280, dtype: object -----------
Epoch 1/100
83/83 - 11s - loss: 1.0623 - mae: 0.8298 - val_loss: 1.0342 - val_mae: 0.8242 - 11s/epoch - 127ms/step
Epoch 2/100
83/83 - 1s - loss: 1.0199 - mae: 0.8058 - val_loss: 1.0080 - val_mae: 0.8068 - 1s/epoch - 12ms/step
Epoch 3/100
83/83 - 1s - loss: 1.0018 - mae: 0.7928 - val_loss: 0.9941 - val_mae: 0.7968 - 989ms/epoch - 12ms/step
Epoch 4/100
83/83 - 1s - loss: 0.9922 - mae: 0.7855 - val_loss: 0.9852 - val_mae: 0.7907 - 1s/epoch - 13ms/step
Epoch 5/100
83/83 - 1s - loss: 0.9861 - mae: 0.7810 - val_loss: 0.9787 - val_mae: 0.7866 - 980ms/epoch - 12ms/step
Epoch 6/100
83/83 - 1s - loss: 0.9814 - mae: 0.7781 - val_loss: 0.9724 - val_mae: 0.7838 - 1s/epoch - 12ms/step
Epoch 7/100
83/83 - 1s - loss: 0.9769 - mae: 0.7758 - val_loss: 0.9660 - val_mae: 0.

...

<ipython-input-125-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-125-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  38726     2018
38727     2018
38728     2018
38729     2018
38730     2018
          ... 
92748     2018
92753     2018
92754     2018
92979     2018
107132    2018
Name: year, Length: 6315, dtype: object -----------
Epoch 1/100
64/64 - 10s - loss: 1.0739 - mae: 0.9095 - val_loss: 1.0551 - val_mae: 0.9068 - 10s/epoch - 162ms/step
Epoch 2/100
64/64 - 1s - loss: 1.0141 - mae: 0.8808 - val_loss: 1.0137 - val_mae: 0.8853 - 1s/epoch - 17ms/step
Epoch 3/100
64/64 - 1s - loss: 0.9776 - mae: 0.8622 - val_loss: 0.9867 - val_mae: 0.8710 - 1s/epoch - 19ms/step
Epoch 4/100
64/64 - 1s - loss: 0.9492 - mae: 0.8482 - val_loss: 0.9650 - val_mae: 0.8607 - 827ms/epoch - 13ms/step
Epoch 5/100
64/64 - 1s - loss: 0.9203 - mae: 0.8352 - val_loss: 0.9437 - val_mae: 0.8518 - 770ms/epoch - 12ms/step
Epoch 6/100
64/64 - 1s - loss: 0.8855 - mae: 0.8209 - val_loss: 0.9169 - val_mae: 0.8412 - 813ms/epoch - 13ms/step
Epoch 7/100
64/64 - 1s - loss: 0.8402 - mae: 0.8003 - val_loss: 0.8824 - val_mae:

<ipython-input-125-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-125-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)


----------year =  38766     2019
38767     2019
38768     2019
38769     2019
38770     2019
          ... 
107282    2019
107283    2019
107284    2019
107495    2019
107496    2019
Name: year, Length: 8013, dtype: object -----------
Epoch 1/100
81/81 - 10s - loss: 1.1530 - mae: 0.9223 - val_loss: 1.0993 - val_mae: 0.9101 - 10s/epoch - 130ms/step
Epoch 2/100
81/81 - 1s - loss: 1.0597 - mae: 0.8813 - val_loss: 1.0308 - val_mae: 0.8741 - 993ms/epoch - 12ms/step
Epoch 3/100
81/81 - 1s - loss: 1.0077 - mae: 0.8554 - val_loss: 0.9912 - val_mae: 0.8523 - 971ms/epoch - 12ms/step
Epoch 4/100
81/81 - 1s - loss: 0.9736 - mae: 0.8384 - val_loss: 0.9585 - val_mae: 0.8358 - 986ms/epoch - 12ms/step
Epoch 5/100
81/81 - 1s - loss: 0.9408 - mae: 0.8237 - val_loss: 0.9229 - val_mae: 0.8205 - 1s/epoch - 14ms/step
Epoch 6/100
81/81 - 1s - loss: 0.9055 - mae: 0.8079 - val_loss: 0.8830 - val_mae: 0.8038 - 1s/epoch - 17ms/step
Epoch 7/100
81/81 - 1s - loss: 0.8647 - mae: 0.7888 - val_loss: 0.8388 - val_mae:

<ipython-input-125-81b9608060d2>:390: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  yearwise_data_test_copy=yearwise_data_test_copy.drop(['temp'],1)
<ipython-input-125-81b9608060d2>:393: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  years_data = years_data.append(yearwise_data_test_copy)
<ipython-input-125-81b9608060d2>:397: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output_df = output_df.append(years_data)
